In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import pi
import math

# importing Qiskit
import qiskit
from qiskit import IBMQ, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

from qiskit.providers.aer.noise import NoiseModel

# import basic plot tools

from qiskit.tools.visualization import plot_histogram, plot_circuit_layout

from qiskit.providers.ibmq import least_busy


import warnings
warnings.filterwarnings('ignore')

In [9]:
# Replace TOKEN with your string
#IBMQ.save_account('0245970821561efcabc18873a763ba09ec3305a9c5b3501ffd1f24f87d26c7e0f55652fb11dd74403ed146b4b20169ace4db11b96abcc13de56e21770f46c606',overwrite=True)
# In subsequent sessios the Token can be loaded from disk
IBMQ.load_account()

Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [10]:
# Plot results
def show_results(D):
    # D is a dictionary with classical bits as keys and count as value
    # example: D = {'000': 497, '001': 527}
    
    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    plt.show()
    #print(D)
    zero = 0
    one = 0
    for r in D:
        if(r[1]=='0'):
            zero += D[r]
        else: 
            one += D[r]
    print("Zero" + str(zero/8192))
    print("One" + str(one/8192))

# Execute circuit, display a histogram of the results
def execute_locally(qc, draw_circuit=False):
    # Compile and run the Quantum circuit on a simulator backend
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim,shots=8192)
    result_sim = job_sim.result()
    result_counts = result_sim.get_counts(qc)
    
    # Print the results
    print("simulation: \n\n", result_counts)
    show_results(result_counts)

In [11]:
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()
physical_devices = provider.backends(simulator=False, operational=True)
for backend in physical_devices:
    print(backend.status())
backend = least_busy(physical_devices)
print (backend);
real_device = backend

BackendStatus(backend_name='ibmqx2', backend_version='2.0.5', operational=True, pending_jobs=2, status_msg='active')
BackendStatus(backend_name='ibmq_16_melbourne', backend_version='2.1.0', operational=True, pending_jobs=4, status_msg='active')
BackendStatus(backend_name='ibmq_vigo', backend_version='1.0.2', operational=True, pending_jobs=3, status_msg='active')
BackendStatus(backend_name='ibmq_ourense', backend_version='1.0.1', operational=True, pending_jobs=1, status_msg='active')
BackendStatus(backend_name='ibmq_london', backend_version='1.1.0', operational=True, pending_jobs=1, status_msg='active')
BackendStatus(backend_name='ibmq_burlington', backend_version='1.1.4', operational=True, pending_jobs=1, status_msg='active')
BackendStatus(backend_name='ibmq_essex', backend_version='1.0.1', operational=True, pending_jobs=2, status_msg='active')
BackendStatus(backend_name='ibmq_armonk', backend_version='1.1.0', operational=True, pending_jobs=0, status_msg='active')
BackendStatus(backend

In [12]:
def generator(qc,qr,cr):
    qc.h(qr[0])
    qc.h(qr[1])
    qc.cx(qr[1],qr[2])
    qc.cx(qr[1],qr[3])

In [13]:
def sign(qc,qr,cr):
    qc.t(qr[0])
    qc.h(qr[0])
    qc.s(qr[0])
    qc.cx(qr[0],qr[1])
    qc.h(qr[0])
    qc.cx(qr[1],qr[2])
    qc.cz(qr[0],qr[2])
    

In [14]:
qc.measure(qr[2],cr[2])

NameError: name 'qc' is not defined